<a name="Exercise"></a>

### Exercises

#### CRUD operations

Create a new db name Todo and a new collection named "CRUD_exercise" and do the following:

**TODO 1**: Take the dict created in the TODO 4 in chapter I and save it in the collection "CRUD_exercise".

**TODO 2**: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

**TODO 3**: Insert the dict created in the TODO 6 Chapter I in the example collection.

**TODO 4**: Get documents where authors key exist in the collection "CRUD_exercise".

**TODO 5**: Change the documents where x = 4 to x = 1.

**TODO 6**: Find documents where author is not_mike and set author as real_mike.

**TODO 7**: Delete documents where author is real_mike.

#### Managing DB

**TODO 8**: create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

**TODO 9**: create a random collection in a random db and put the new collection in the tutorial DB

**TODO 10**: What is the difference between an inner join and an outer join ? Is the query seen during course an inner or outer join ? Play with the query to show all the joins.

#### Real world problems

**TODO 11**:  Use the oaipmh and api code get papers after January 2020 and for "cs,math,econ" categories. Insert them in MongoDB. Import only the first 200. How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "cs".

**TODO 12**: Do the same as TODO 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.

**TODO 13**: Download a random image and store it in a collection.

**TODO 14**: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)

**TODO 15**: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.

**TODO 16**: Download a [sound sample](https://freesound.org/browse/). Try to store it in MongoDB 

**TODO 17**: Create a collection with 30M observation with a single key : "year" which is a random value between 2000-2020. Get documents with year = 2000. Does using an index helps ? 




SQLite:

In [1]:

#1: Read all the data from bc.csv, how many rows is there ? Columns ? 

import pandas as pd

df =pd.read_csv("data/chap1/bc.csv")
li, col = df.shape
print("Ce DF contient", li,"lignes et", col,"colonnes")
df

Ce DF contient 569 lignes et 33 colonnes


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [3]:
#2: Insert the bc.csv in the sqlite format.
import sqlite3

conn = sqlite3.connect('/Users/valen/Desktop/SAS/BC.db')
c = conn.cursor()



In [4]:
#3: In the same db create a new table called tomatch with two variables id_ and rdm_float. 
c.execute("""CREATE TABLE tomatch3 (
          id INT PRIMARY KEY,
          rdm_float VARCHAR(100)
) """)

conn.commit()

OperationalError: table tomatch3 already exists

In [5]:
#4: In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.
import tqdm
import random as rd

query = """INSERT INTO tomatch3(id, rdm_float) VALUES (?,?)"""

list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,1)):
    values = (i,rd.random())
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        list_of_insertion = []
conn.commit()

  0%|          | 9999/9950000 [00:00<00:08, 1206005.63it/s]


IntegrityError: UNIQUE constraint failed: tomatch3.id

In [7]:
#**5**: Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.
# Execute the corrected SQL query
c.execute("""
CREATE TABLE Newtomatch AS
SELECT 
    AVG(bc.texture_worst) AS average_value,
    COUNT(bc.texture_worst) AS count_value,
    MAX(bc.texture_worst) AS max_value,
    MIN(bc.texture_worst) AS min_value 
FROM 
    bc
INNER JOIN 
    tomatch3 ON bc.id = tomatch3.id
""")

c.fetchone()

OperationalError: table Newtomatch already exists

In [9]:
c.execute("SELECT * FROM Newtomatch")

In [8]:
#**6**: How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

c.execute("""
SELECT COUNT(*) 
FROM Newtomatch WHERE radius_mean > 15
""")
total_radius_greater_15 = c.fetchone()[0]

c.execute("""
SELECT COUNT(*) 
FROM Newtomatch WHERE radius_mean > 15 AND texture_mean > 20
""")
total_radius_and_texture = c.fetchone()[0]

print(f"total_radius_greater_15 : {total_radius_greater_15}")
print(f"total_radius_and_texture: {total_radius_and_texture}")

OperationalError: no such column: radius_mean

In [13]:
#**7**: For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups
c.execute("""
SELECT diagnosis,
    ROUND(AVG(radius_mean), 2) AS average_radius
FROM bc GROUP BY diagnosis
""")
results = c.fetchall()

if len(results) == 2:  
    difference = abs(results[0][1] - results[1][1])
    print(f"Mean Radius for {results[0][0]}: {results[0][1]}")
    print(f"Mean Radius for {results[1][0]}: {results[1][1]}")
    print(f"Difference between means: {difference:.2f}")
else:
    print("Unexpected number of diagnosis categories.")

Mean Radius for B: 12.15
Mean Radius for M: 17.46
Difference between means: 5.31


In [14]:
#**8**: Change the name of the column "diagnosis" to "label"
c.execute("""
ALTER TABLE bc
RENAME COLUMN diagnosis TO label
""")
conn.commit()

In [16]:
#**9**: Add a column named "day" of type DATE.
c.execute("""
ALTER TABLE bc
ADD COLUMN day DATE
""")
conn.commit()

OperationalError: duplicate column name: day

In [18]:
#**10**: Change the type of the column "area_mean" to int (hint,if not possible on sqlite, create a new column)
c.execute("""
ALTER TABLE bc
ADD COLUMN area_mean_int INTEGER
""")
conn.commit()
c.execute("""
UPDATE bc
SET area_mean_int = CAST(area_mean AS INTEGER)
""")
conn.commit()

In [22]:
#**11**: create a new table called tomatch6 exactly the same way as tomatch3 except you increment id_ by 10.
c.execute("""
CREATE TABLE tomatch6 (
    id INTEGER PRIMARY KEY,
    rdm_float VARCHAR(100)
)
""")

conn.commit()
last_id = 10 
increment = 10 

c.execute("SELECT MAX(id) FROM tomatch6")
result = c.fetchone()
if result[0] is not None:
    last_id = result[0] + increment

c.execute("INSERT INTO tomatch6 (id, rdm_float) VALUES (?, ?)", (last_id, 'Example value'))
conn.commit()

In [10]:
#**12**: Do an inner join of bc, tomatch3 and tomatch6 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch3 AND tomatch6.

c.execute("""
CREATE TABLE test AS
SELECT
    bc.*, tomatch3.rdm_float AS rdm_float3, tomatch6.rdm_float AS rdm_float6
FROM 
    bc
INNER JOIN 
    tomatch3 ON bc.id = tomatch3.id
INNER JOIN 
    tomatch6 ON bc.id = tomatch6.id
WHERE 
    bc.radius_mean > 15 AND 
    tomatch3.rdm_float > 0.50 AND 
    tomatch6.rdm_float > 0.50
""")
results = c.fetchall()

MySQL

In [ ]:
#**1**: How many orders were placed in total?


In [ ]:
#**2**: What is the total sales amount?

In [ ]:
#**3**: What is the average profit per order?

In [ ]:
#**4**: Which customer has placed the most orders?


In [ ]:
#**5**: What is the total profit for each product category?


In [ ]:
#**6**: What is the average discount percentage?


In [ ]:
#**7**: Which region has the highest total sales?


In [ ]:
#**8**: How many orders were shipped using each shipping mode?


In [ ]:
#**9**: What is the total quantity ordered for each product sub-category?


In [ ]:
#**10**: Which product has the highest profit margin?


In [ ]:
#**11**: What is the total sales amount for each year?


In [ ]:
#**12**: Which customer segment has the highest average sales?


PostgreSQL

In [11]:
#**1**: What is the average age of the customers?

In [ ]:
#**2**: How many customers have defaulted on loans?

In [ ]:
#**3**: What is the maximum balance in the account?


In [ ]:
#**4**: How many customers have a housing loan?


In [ ]:
#**5**: What is the education level of the youngest customer?


In [ ]:
#**6**: What is the marital status of the oldest customer?


In [ ]:
#**7**: What is the distribution of the months in which customers were last contacted?


In [ ]:
#**8**: What is the average duration of contact for customers who have a personal loan?


In [ ]:
#**9**: How many customers were previously contacted, and what was the average duration of their last contact?


In [ ]:
#**10**: What is the distribution of job types among customers?


In [ ]:
#**11**: How many customers have been contacted more than once in the current campaign?


In [ ]:
#**12**: What is the success rate of previous marketing campaigns (poutcome) among customers